# Clase 2 — Técnicas avanzadas en Ciencia de Datos

---

# Resumen ejecutivo y objetivos de la sesión

Duración: **180 minutos** (3 horas)

Objetivos de aprendizaje:

1. Comprender matemáticamente y conceptualmente PCA y t-SNE; conocer sus ventajas, limitaciones y pautas prácticas.
2. Aplicar PCA y t-SNE a datos reales (scikit-learn `digits` / `iris`) siguiendo buenas prácticas de ingeniería reproducible.
3. Crear visualizaciones interactivas con **Plotly** y un pequeño **Dash app** para explorar embeddings y parámetros en tiempo real.
4. Integrar normas de estilo y calidad de código (PEP-8, docstrings, typing, pipelines reproducibles).

Público: estudiantes con conocimientos previos de Python, pandas y scikit-learn básico.

Requisitos previos para los alumnos:

* Python 3.9+ instalado.
* Paquetes: `numpy, pandas, scikit-learn, plotly, dash` (lista más abajo con versiones sugeridas).

---

# Materiales y setup (comandos rápidos)

requirements.txt:

```
numpy>=1.23
pandas>=1.5
scikit-learn>=1.2
plotly>=5.15
dash>=2.9
umap-learn>=0.5  # opcional, alternativa a t-SNE
```
---

# CONTENIDO TEÓRICO 



# PCA (Principal Component Analysis)

## 1. Definición general

**PCA (Análisis de Componentes Principales)** es una técnica estadística de **reducción de dimensionalidad lineal** que busca transformar un conjunto de variables posiblemente correlacionadas en un conjunto más pequeño de variables **no correlacionadas**, llamadas **componentes principales**.

Cada componente principal es una **combinación lineal de las variables originales**, y están ordenados de tal manera que:

* El primer componente captura la **máxima varianza posible** de los datos.
* El segundo componente captura la **máxima varianza posible no explicada por el primero**, y así sucesivamente.
* Todos los componentes son **ortogonales** entre sí.

---

## 2. Objetivos principales

* **Reducir la dimensionalidad**: Mantener la mayor parte de la información (varianza) eliminando redundancia.
* **Eliminar multicolinealidad**: Los componentes principales no están correlacionados.
* **Mejorar eficiencia**: Modelos de Machine Learning entrenan más rápido en espacios reducidos.
* **Facilitar visualización**: Transformar datos de alta dimensión a 2D o 3D para graficarlos.

---

## 3. Fundamentos matemáticos

Dado un dataset con $n$ observaciones y $p$ variables:

1. **Estandarización**
   Se recomienda escalar las variables ($z$-score) para que todas contribuyan en la misma magnitud:

   $$
   z_{ij} = \frac{x_{ij} - \bar{x}_j}{s_j}
   $$

   donde $\bar{x}_j$ es la media y $s_j$ la desviación estándar de la variable $j$.

---

2. **Matriz de covarianza o correlación**
   Se calcula la matriz de covarianza $\Sigma$:

   $$
   \Sigma = \frac{1}{n-1} Z^T Z
   $$

   donde $Z$ es la matriz de datos estandarizados.

   Esta matriz refleja cómo varían conjuntamente las variables.

---

3. **Descomposición espectral (Eigen decomposition)**
   Se encuentran **autovalores** ($\lambda$) y **autovectores** ($v$) de $\Sigma$:

   $$
   \Sigma v = \lambda v
   $$

   * Los **autovalores** indican la varianza explicada por cada componente.
   * Los **autovectores** son las direcciones de los componentes principales.

   **Ordenar** los autovalores de mayor a menor → así se definen los componentes principales.

---

4. **Varianza explicada**
   La proporción de varianza capturada por el $k$-ésimo componente es:

   $$
   \text{Varianza explicada}_k = \frac{\lambda_k}{\sum_{i=1}^p \lambda_i}
   $$

   Generalmente se seleccionan los primeros componentes que acumulen, por ejemplo, el **95% de la varianza**.

---

5. **Transformación de los datos**
   La proyección de los datos originales sobre los nuevos ejes:

   $$
   Z_{\text{PCA}} = Z \cdot V_k
   $$

   donde $V_k$ son los primeros $k$ autovectores.

---

## 4. Interpretación práctica

* **Primer componente (PC1):** Dirección que maximiza la varianza.
* **Segundo componente (PC2):** Ortogonal a PC1, maximiza varianza restante.
* **Carga (loading):** Coeficiente de cada variable en la combinación lineal; indica qué variables influyen más en el componente.
* **Score:** Coordenadas de cada observación en el nuevo espacio de componentes.

---

## 5. Ventajas

* Reduce ruido y redundancia en los datos.
* Facilita visualización de datasets complejos en 2D o 3D.
* Ayuda a mejorar el rendimiento de modelos en datasets muy grandes.

---

## 6. Limitaciones

* Es un método **lineal** — no captura relaciones no lineales.
* Los componentes no siempre son interpretables fácilmente (son combinaciones lineales abstractas).
* Sensible a escalado: si no se estandarizan las variables, aquellas con valores más grandes dominan el análisis.

---

## 7. Ejemplo ilustrativo (caso intuitivo)

Imagina un dataset con dos variables altamente correlacionadas (ejemplo: **altura** y **peso** en una población).

* PCA encuentra una **nueva variable** (componente principal) que resume esa correlación (dirección de máxima varianza).
* En lugar de trabajar con 2 variables redundantes, podemos proyectar los datos en ese componente principal y **resumir la información en una sola variable**.

---


# t-SNE (t-distributed Stochastic Neighbor Embedding)

## 1. Definición general

El **t-SNE** es una técnica de **reducción de dimensionalidad no lineal** diseñada principalmente para **visualización** de datos de alta dimensión en 2D o 3D.
Fue propuesta por Laurens van der Maaten y Geoffrey Hinton en 2008 como una mejora sobre **SNE (Stochastic Neighbor Embedding)**.

* Su objetivo: **preservar la estructura local de los datos** (vecindades) en un espacio de baja dimensión.

A diferencia de **PCA**, que es lineal y global, t-SNE busca que **puntos similares en alta dimensión se mantengan cercanos en la proyección**.

---

## 2. Intuición

* Imagina un dataset en 100 dimensiones.
* t-SNE "aprende" una representación en 2D donde **puntos similares** (según distancia en alta dimensión) siguen siendo vecinos.
* Se enfoca en la **estructura local**, no en mantener las distancias globales.

Ejemplo:

* Si tienes imágenes de dígitos (0-9), t-SNE agrupa los dígitos iguales cerca entre sí, aunque la distancia entre los diferentes dígitos no sea perfectamente proporcional a la original.

---

## 3. Parámetros clave

* **Perplexity**: controla el balance entre estructura local y global (típicamente entre 5 y 50). Relacionado con el número efectivo de vecinos.
* **Learning rate**: afecta la estabilidad de la optimización; valores muy bajos → agrupamiento artificial, muy altos → dispersión.
* **n\_iter**: número de iteraciones; demasiadas pocas → no converge, demasiadas → sobreajuste.
* **init**: inicialización, comúnmente con PCA para mayor estabilidad.
* **random\_state**: controla la reproducibilidad.

---

## 4. Ventajas

* Excelente para visualizar datos complejos en 2D o 3D.
* Captura relaciones locales muy bien.
* Agrupa naturalmente clusters.

---

## 5. Limitaciones

* **No es determinista** (resultados varían salvo que fijes `random_state`).
* **No preserva relaciones globales**: la distancia entre clusters no siempre es significativa.
* **Computacionalmente costoso** en datasets grandes.
* **No es transformador general**: no puedes aplicar el mismo embedding a nuevos datos sin recalcular.

---

## 6. Buenas prácticas

* Aplicar **PCA previo** para reducir dimensiones a 30-50 antes de t-SNE → mejora velocidad y estabilidad.
* Probar varios valores de `perplexity` y comparar resultados.
* Usar t-SNE solo como herramienta exploratoria/visual, no como método para modelado predictivo.
* Para datasets grandes (>50k muestras), considerar **UMAP** como alternativa más rápida y escalable.

---

## 7. Ejemplo intuitivo

Si tienes un dataset de imágenes de ropa (camisas, zapatos, bolsos), t-SNE puede proyectar los datos en 2D de forma que cada tipo de prenda quede agrupada en clusters separados, **sin que tú hayas dado las etiquetas previamente**.

Esto lo convierte en una herramienta poderosa para **descubrimiento de patrones** y **detección de estructuras ocultas**.

---


## Alternativa: UMAP

* UMAP es otra técnica no lineal, más rápida y que suele preservar mejor la estructura global; buena alternativa a t-SNE. ([arXiv][5])

---


# Enunciado del Ejercicio: Reducción de Dimensionalidad con PCA y t-SNE

## Contexto

Las organizaciones actuales generan grandes volúmenes de información de diversa naturaleza: transacciones financieras, registros médicos, datos de sensores, imágenes, textos, etc. Sin embargo, estos datos suelen tener **alta dimensionalidad** (muchas variables o características), lo cual dificulta su análisis, interpretación y visualización.

En el campo de la **Ciencia de Datos y Big Data**, las técnicas de **reducción de dimensionalidad** permiten transformar datasets complejos en representaciones más compactas y fáciles de interpretar, sin perder demasiada información. Entre estas técnicas se encuentran:

* **PCA (Principal Component Analysis):** Método lineal que transforma las variables originales en nuevas combinaciones (componentes principales) maximizando la varianza explicada.
* **t-SNE (t-distributed Stochastic Neighbor Embedding):** Método no lineal que preserva relaciones de vecindad, muy útil para la visualización de datos complejos en 2D o 3D.

Este ejercicio busca que los estudiantes apliquen estas dos técnicas sobre un dataset real y comparen los resultados, entendiendo sus ventajas y limitaciones.

---

## Objetivo del Ejercicio

1. Comprender la importancia de la reducción de dimensionalidad en ciencia de datos.
2. Aplicar **PCA** y **t-SNE** a un dataset real de dígitos manuscritos.
3. Evaluar y comparar cómo ambas técnicas representan los datos en un espacio de dos dimensiones.
4. Visualizar los resultados mediante gráficos interactivos que faciliten la interpretación.

---

## Dataset

Se trabajará con el dataset **Digits** de `sklearn`, que contiene:

* **1797 imágenes de dígitos manuscritos (0 a 9).**
* Cada imagen es de **8x8 píxeles**, lo que genera **64 variables**.
* La etiqueta asociada a cada observación indica el número real escrito.

---

## Actividades a realizar

### **Parte 1: Preparación y preprocesamiento**

1. Cargar el dataset `Digits` desde `sklearn.datasets`.
2. Revisar la forma y las características de los datos.
3. Normalizar las variables utilizando `StandardScaler` para garantizar que todas las características tengan la misma importancia.

---

### **Parte 2: Análisis con PCA**

1. Reducir las 64 dimensiones a 2 componentes principales con **PCA**.
2. Calcular e interpretar la **varianza explicada** por cada componente.
3. Generar un gráfico de dispersión en 2D donde cada punto represente un dígito, coloreado según su etiqueta (0–9).
4. Responder:

   * ¿Qué porcentaje de información se conserva en los 2 primeros componentes?
   * ¿Se observan grupos definidos en el gráfico?

---

### **Parte 3: Análisis con t-SNE**

1. Aplicar **t-SNE** para reducir las dimensiones a 2D.
2. Configurar los parámetros clave:

   * `perplexity = 30`
   * `learning_rate = 200`
   * `n_iter = 1000`
3. Visualizar los resultados en un gráfico de dispersión coloreado por etiquetas.
4. Responder:

   * ¿Qué diferencias se observan frente al PCA?
   * ¿Qué ventajas ofrece t-SNE en este caso?

---

### **Parte 4: Comparación y Conclusiones**

1. Comparar visualmente los resultados de **PCA** y **t-SNE**.
2. Discutir en grupo:

   * ¿Cuál de las dos técnicas genera una separación más clara de los dígitos?
   * ¿En qué contextos empresariales podría aplicarse cada técnica (ejemplo: segmentación de clientes, análisis de sensores, reconocimiento de imágenes, etc.)?
3. Elaborar un breve informe (máximo 1 página) con conclusiones sobre:

   * La utilidad de PCA.
   * La utilidad de t-SNE.
   * Sus principales diferencias.

---

## Productos esperados

* Código en Python bien estructurado y comentado.
* Gráficos de dispersión comparativos de PCA y t-SNE.
* Respuestas a las preguntas planteadas.
* Informe de conclusiones.

---


In [ ]:
"""
Ejemplo práctico: PCA y t-SNE en Ciencia de Datos
-------------------------------------------------
Este script aplica reducción de dimensionalidad utilizando:
1. PCA (Principal Component Analysis).
2. t-SNE (t-distributed Stochastic Neighbor Embedding).

Dataset: dígitos manuscritos de sklearn (8x8 píxeles, 64 variables).
Objetivo: visualizar cómo se distribuyen los datos en espacios reducidos.

Buenas prácticas aplicadas:
- Código comentado siguiendo PEP 8.
- Uso de funciones para modularizar el análisis.
- Reproducibilidad asegurada con random_state.
- Manejo de errores y validación de entrada.
- Parámetros configurables y métricas de rendimiento.
"""

# =========================
# Importación de librerías
# =========================
# Librerías estándar de Python
import time                    # Para medir tiempo de ejecución
import warnings               # Para mostrar advertencias
from typing import Tuple, Optional  # Para anotaciones de tipo

# Librerías de terceros para análisis de datos y visualización
import matplotlib.pyplot as plt    # Para crear gráficos
import numpy as np                 # Para operaciones numéricas
import seaborn as sns             # Para gráficos estadísticos más bonitos
from sklearn.datasets import load_digits      # Dataset de dígitos manuscritos
from sklearn.decomposition import PCA         # Algoritmo PCA
from sklearn.manifold import TSNE            # Algoritmo t-SNE
from sklearn.preprocessing import StandardScaler  # Para normalizar datos

# =========================
# Constantes de configuración
# =========================
RANDOM_STATE = 42              # Semilla para reproducibilidad
DEFAULT_PERPLEXITY = 30        # Perplejidad por defecto para t-SNE
DEFAULT_LEARNING_RATE = 200    # Tasa de aprendizaje por defecto para t-SNE
DEFAULT_N_ITER = 1000          # Número de iteraciones por defecto para t-SNE
DEFAULT_FIGURE_SIZE = (8, 6)   # Tamaño por defecto de las figuras (ancho, alto)

# =========================
# Funciones auxiliares
# =========================
def validar_datos(X, y=None):
    """
    Valida los datos de entrada para las operaciones de reducción de dimensionalidad.
    :param X: Datos de entrada (matriz de características).
    :param y: Etiquetas opcionales.
    :return: True si los datos son válidos.
    """
    # Verificar que los datos no estén vacíos
    if X is None or len(X) == 0:
        raise ValueError("Los datos de entrada no pueden estar vacíos.")
    
    # Convertir a array de numpy si no lo es
    if not isinstance(X, np.ndarray):
        X = np.array(X)
    
    # Verificar que sea una matriz 2D (muestras x características)
    if len(X.shape) != 2:
        raise ValueError("Los datos deben ser una matriz 2D.")
    
    # Verificar que las etiquetas coincidan con el número de muestras
    if y is not None and len(y) != len(X):
        raise ValueError("El número de etiquetas debe coincidir con el número de muestras.")
    
    return True  # Si llegamos aquí, los datos son válidos


def aplicar_pca(X, n_componentes=2):
    """
    Aplica PCA para reducir dimensionalidad.
    :param X: Datos de entrada (matriz de características).
    :param n_componentes: Número de componentes principales.
    :return: Tupla con (datos transformados, tiempo de ejecución).
    """
    try:
        # Validar que los datos de entrada sean correctos
        validar_datos(X)
        
        # Verificar que no se pidan más componentes que características disponibles
        if n_componentes > X.shape[1]:
            warnings.warn(f"n_componentes ({n_componentes}) mayor que el número de características ({X.shape[1]}). "
                         f"Se ajustará a {X.shape[1]}.")
            n_componentes = X.shape[1]  # Ajustar al máximo posible
        
        # Iniciar medición de tiempo
        inicio = time.time()
        
        # Crear objeto PCA con el número de componentes especificado
        pca = PCA(n_components=n_componentes, random_state=RANDOM_STATE)
        
        # Aplicar PCA: ajustar el modelo y transformar los datos
        X_pca = pca.fit_transform(X)
        
        # Calcular tiempo de ejecución
        tiempo_ejecucion = time.time() - inicio
        
        # Mostrar información sobre la varianza explicada
        print(f"Varianza explicada por cada componente: {pca.explained_variance_ratio_}")
        print(f"Varianza total explicada: {sum(pca.explained_variance_ratio_):.4f}")
        print(f"Tiempo de ejecución PCA: {tiempo_ejecucion:.2f} segundos")
        
        # Retornar datos transformados y tiempo de ejecución
        return X_pca, tiempo_ejecucion
        
    except Exception as e:
        # Si hay algún error, mostrarlo y re-lanzarlo
        print(f"Error en PCA: {e}")
        raise


def aplicar_tsne(X, n_componentes=2, perplejidad=DEFAULT_PERPLEXITY, 
                learning_rate=DEFAULT_LEARNING_RATE, n_iter=DEFAULT_N_ITER):
    """
    Aplica t-SNE para reducción de dimensionalidad.
    :param X: Datos de entrada (matriz de características).
    :param n_componentes: Dimensiones objetivo.
    :param perplejidad: Balance entre densidad local/global.
    :param learning_rate: Velocidad de aprendizaje.
    :param n_iter: Número de iteraciones.
    :return: Tupla con (datos transformados, tiempo de ejecución).
    """
    try:
        # Validar que los datos de entrada sean correctos
        validar_datos(X)
        
        # Validar perplejidad: debe ser menor que el número de muestras
        if perplejidad >= len(X):
            perplejidad = min(perplejidad, len(X) - 1)  # Ajustar al máximo permitido
            warnings.warn(f"Perplejidad ajustada a {perplejidad} (máximo permitido: {len(X) - 1})")
        
        # Iniciar medición de tiempo
        inicio = time.time()
        
        # Crear objeto t-SNE con los parámetros especificados
        tsne = TSNE(
            n_components=n_componentes,      # Número de dimensiones de salida
            perplexity=perplejidad,          # Controla el balance local/global
            learning_rate=learning_rate,     # Velocidad de optimización
            random_state=RANDOM_STATE,       # Para reproducibilidad
            n_iter=n_iter                    # Número de iteraciones de optimización
        )
        
        # Aplicar t-SNE: ajustar el modelo y transformar los datos
        X_tsne = tsne.fit_transform(X)
        
        # Calcular tiempo de ejecución
        tiempo_ejecucion = time.time() - inicio
        
        # Mostrar tiempo de ejecución
        print(f"Tiempo de ejecución t-SNE: {tiempo_ejecucion:.2f} segundos")
        
        # Retornar datos transformados y tiempo de ejecución
        return X_tsne, tiempo_ejecucion
        
    except Exception as e:
        # Si hay algún error, mostrarlo y re-lanzarlo
        print(f"Error en t-SNE: {e}")
        raise


def graficar_resultados(X_transformado, y, titulo, figura_size=DEFAULT_FIGURE_SIZE):
    """
    Genera un gráfico de dispersión coloreado por etiqueta.
    :param X_transformado: Datos reducidos.
    :param y: Etiquetas de clase.
    :param titulo: Título del gráfico.
    :param figura_size: Tamaño de la figura (ancho, alto).
    """
    try:
        # Validar que los datos de entrada sean correctos
        validar_datos(X_transformado, y)
        
        # Obtener el número de componentes (dimensiones) de los datos transformados
        n_componentes = X_transformado.shape[1]
        
        if n_componentes == 2:
            # Visualización 2D: gráfico de dispersión simple
            plt.figure(figsize=figura_size)  # Crear nueva figura con tamaño especificado
            sns.scatterplot(
                x=X_transformado[:, 0],      # Eje X: primera componente
                y=X_transformado[:, 1],      # Eje Y: segunda componente
                hue=y,                       # Colorear por etiqueta de clase
                palette="tab10",             # Paleta de colores para las clases
                legend="full",               # Mostrar leyenda completa
                alpha=0.7                    # Transparencia de los puntos
            )
            plt.title(titulo, fontsize=14)   # Título del gráfico
            plt.xlabel("Componente 1")       # Etiqueta del eje X
            plt.ylabel("Componente 2")       # Etiqueta del eje Y
            plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')  # Posición de la leyenda
            
        elif n_componentes == 3:
            # Visualización 3D: gráfico de dispersión tridimensional
            fig = plt.figure(figsize=figura_size)  # Crear figura
            ax = fig.add_subplot(111, projection='3d')  # Agregar subplot 3D
            
            # Crear gráfico de dispersión 3D
            scatter = ax.scatter(
                X_transformado[:, 0],        # Coordenada X
                X_transformado[:, 1],        # Coordenada Y
                X_transformado[:, 2],        # Coordenada Z
                c=y,                         # Colorear por etiqueta
                cmap='tab10',                # Mapa de colores
                alpha=0.7                    # Transparencia
            )
            ax.set_title(titulo, fontsize=14)  # Título
            ax.set_xlabel("Componente 1")      # Etiqueta eje X
            ax.set_ylabel("Componente 2")      # Etiqueta eje Y
            ax.set_zlabel("Componente 3")      # Etiqueta eje Z
            
        else:
            # Para más de 3 dimensiones, mostrar solo las primeras 2
            plt.figure(figsize=figura_size)
            sns.scatterplot(
                x=X_transformado[:, 0],      # Primera componente
                y=X_transformado[:, 1],      # Segunda componente
                hue=y,                       # Colorear por clase
                palette="tab10",
                legend="full",
                alpha=0.7
            )
            # Título indicando que solo se muestran 2 de N componentes
            plt.title(f"{titulo} (primeras 2 componentes de {n_componentes})", fontsize=14)
            plt.xlabel("Componente 1")
            plt.ylabel("Componente 2")
            plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
        
        plt.tight_layout()  # Ajustar layout para evitar solapamientos
        plt.show()          # Mostrar el gráfico
        
    except Exception as e:
        # Si hay algún error, mostrarlo y re-lanzarlo
        print(f"Error en visualización: {e}")
        raise


# =========================
# Flujo principal del script
# =========================
def main():
    """
    Función principal que ejecuta el análisis de reducción de dimensionalidad.
    """
    try:
        # Mostrar encabezado del análisis
        print("=" * 50)
        print("ANÁLISIS DE REDUCCIÓN DE DIMENSIONALIDAD")
        print("=" * 50)
        
        # 1. Cargar dataset de dígitos manuscritos
        print("\n1. Cargando dataset de dígitos manuscritos...")
        digits = load_digits()        # Cargar dataset de sklearn
        X = digits.data               # Matriz de características (1797 x 64)
        y = digits.target             # Vector de etiquetas (1797,)

        # Mostrar información básica del dataset
        print(f"Dimensiones originales: {X.shape}")  # 1797 muestras, 64 variables
        print(f"Número de clases: {len(np.unique(y))}")  # 10 clases (dígitos 0-9)

        # 2. Escalado de datos (normalización estándar)
        print("\n2. Escalando datos...")
        scaler = StandardScaler()     # Crear objeto escalador
        X_scaled = scaler.fit_transform(X)  # Ajustar y transformar datos
        print("Escalado completado.")

        # 3. Aplicar PCA (Análisis de Componentes Principales)
        print("\n3. Aplicando PCA...")
        X_pca, tiempo_pca = aplicar_pca(X_scaled, n_componentes=2)  # Reducir a 2D
        graficar_resultados(X_pca, y, "Visualización con PCA")      # Mostrar gráfico

        # 4. Aplicar t-SNE (t-distributed Stochastic Neighbor Embedding)
        print("\n4. Aplicando t-SNE...")
        X_tsne, tiempo_tsne = aplicar_tsne(X_scaled, n_componentes=2, 
                                          perplejidad=DEFAULT_PERPLEXITY, 
                                          learning_rate=DEFAULT_LEARNING_RATE)
        graficar_resultados(X_tsne, y, "Visualización con t-SNE")   # Mostrar gráfico

        # 5. Mostrar resumen de rendimiento
        print("\n" + "=" * 50)
        print("RESUMEN DE RENDIMIENTO")
        print("=" * 50)
        print(f"Tiempo total PCA: {tiempo_pca:.2f} segundos")
        print(f"Tiempo total t-SNE: {tiempo_tsne:.2f} segundos")
        print(f"Tiempo total: {tiempo_pca + tiempo_tsne:.2f} segundos")
        print("\nAnálisis completado exitosamente.")
        
    except Exception as e:
        # Si hay algún error, mostrarlo y re-lanzarlo
        print(f"Error en el análisis principal: {e}")
        raise


# Punto de entrada del script
if __name__ == "__main__":
    main()  # Ejecutar función principal solo si se ejecuta directamente


# Enunciado del ejercicio

## Tema: Reducción de Dimensionalidad Avanzada con PCA y t-SNE

En esta práctica los estudiantes explorarán dos técnicas clave de reducción de dimensionalidad en Ciencia de Datos:

1. **PCA (Principal Component Analysis)** → Técnica lineal que transforma los datos en componentes ortogonales maximizando la varianza explicada.
2. **t-SNE (t-distributed Stochastic Neighbor Embedding)** → Técnica no lineal que preserva relaciones de vecindad y es muy útil para la visualización en 2D/3D de datos complejos.

Se trabajará con datasets disponibles en **scikit-learn**:

* **Digits** (1797 imágenes de dígitos manuscritos, 64 variables).
* **Iris** (150 muestras, 4 variables, dataset pequeño para comparación).

El objetivo es:

* Entender cómo se reducen datos de alta dimensionalidad.
* Comparar PCA vs t-SNE en visualización y agrupamiento.
* Generar **gráficos interactivos** con **Plotly**.
* Implementar un **Dashboard en Dash** para experimentar con parámetros.

---

# Estructura de carpetas recomendada

```bash
dimred_project/
├── README.md                 # Descripción del proyecto y objetivos
├── requirements.txt          # Librerías necesarias
├── example_dimred.py         # Script principal (PCA y t-SNE con Plotly)
├── dash_dimred_app.py        # Dashboard interactivo con Dash
└── notebooks/
    └── demo_dimred.ipynb     # Notebook para clase (explicación paso a paso)
```

* Esto da claridad:

* `example_dimred.py` → código profesional reproducible.
* `dash_dimred_app.py` → aplicación web simple para exploración interactiva.
* `notebooks/` → versión educativa y explicada para los estudiantes.

---




# Archivo `requirements.txt`

In [ ]:
dash==2.17.1
plotly==5.24.1
scikit-learn==1.5.2
pandas==2.2.3
numpy==2.1.2


# Código principal: `example_dimred.py`

In [ ]:
"""
Ejemplo: PCA y t-SNE con visualizaciones Plotly
------------------------------------------------
Este script aplica reducción de dimensionalidad usando PCA y t-SNE
en datasets de sklearn (Digits, Iris). Incluye:
- Buenas prácticas (PEP8, typing, docstrings).
- Logging para trazabilidad.
- Gráficos interactivos con Plotly.
"""

from __future__ import annotations
import logging
from dataclasses import dataclass
from typing import Tuple

import numpy as np
import pandas as pd
from sklearn.datasets import load_digits, load_iris
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import plotly.express as px

# =====================
# Configuración logging
# =====================
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


# =====================
# Definición de Dataset
# =====================
@dataclass
class Dataset:
    X: np.ndarray
    y: np.ndarray
    feature_names: list[str]
    target_names: list[str]


def load_dataset(name: str = "digits") -> Dataset:
    """
    Carga dataset de sklearn: 'digits' o 'iris'.
    """
    if name == "digits":
        data = load_digits()
    elif name == "iris":
        data = load_iris()
    else:
        raise ValueError("Dataset no soportado. Usa 'digits' o 'iris'.")

    X = data["data"]
    y = data["target"]
    feature_names = list(data.get("feature_names", [f"f{i}" for i in range(X.shape[1])]))
    target_names = list(map(str, data.get("target_names", np.unique(y).astype(str))))
    logger.info("Dataset %s cargado: X=%s, y=%s", name, X.shape, y.shape)
    return Dataset(X=X, y=y, feature_names=feature_names, target_names=target_names)


# =====================
# Funciones PCA y t-SNE
# =====================
def compute_pca(X: np.ndarray,
                n_components: int = 2,
                random_state: int | None = 42) -> Tuple[np.ndarray, PCA]:
    """
    Escala datos y aplica PCA. Retorna embedding y el objeto PCA.
    """
    pipeline = Pipeline([
        ("scaler", StandardScaler()),
        ("pca", PCA(n_components=n_components, random_state=random_state))
    ])
    X_trans = pipeline.fit_transform(X)
    pca_obj: PCA = pipeline.named_steps["pca"]
    logger.info("PCA varianza explicada: %s", pca_obj.explained_variance_ratio_[:5])
    return X_trans, pca_obj


def compute_tsne(X: np.ndarray,
                 n_components: int = 2,
                 perplexity: float = 30.0,
                 learning_rate: float = 200.0,
                 random_state: int | None = 42,
                 init_shape: str = "pca") -> np.ndarray:
    """
    Aplica t-SNE (recomendado escalar y usar init='pca').
    """
    scaler = StandardScaler()
    Xs = scaler.fit_transform(X)

    tsne = TSNE(
        n_components=n_components,
        perplexity=perplexity,
        learning_rate=learning_rate,
        init=init_shape,
        random_state=random_state,
        n_iter=1000
    )
    embedding = tsne.fit_transform(Xs)
    logger.info("t-SNE terminado: embedding %s", embedding.shape)
    return embedding


# =====================
# Funciones de gráficos
# =====================
def plot_embedding(df: pd.DataFrame, title: str = "Embedding 2D") -> None:
    """
    Gráfico scatter interactivo con Plotly Express.
    """
    fig = px.scatter(df, x="x", y="y", color="label",
                     hover_data=df.columns.tolist(), title=title)
    fig.show()


# =====================
# Flujo demo
# =====================
def demo_flow():
    """Flujo demo para ejecutar en clase."""
    ds = load_dataset("digits")
    X, y = ds.X, ds.y

    # PCA 2D
    X_pca2, pca2 = compute_pca(X, n_components=2)
    df_pca = pd.DataFrame({"x": X_pca2[:, 0], "y": X_pca2[:, 1], "label": y})
    plot_embedding(df_pca, title="Digits - PCA (2D)")

    # Varianza explicada
    X_pca_full, pca_full = compute_pca(X, n_components=min(30, X.shape[1]))
    explained = pca_full.explained_variance_ratio_
    df_ev = pd.DataFrame({"component": range(1, len(explained) + 1),
                          "explained_variance": explained})
    px.bar(df_ev, x="component", y="explained_variance",
           title="Explained variance per component").show()

    # t-SNE
    emb_tsne = compute_tsne(X, n_components=2, perplexity=30, learning_rate=200, init_shape="pca")
    df_tsne = pd.DataFrame({"x": emb_tsne[:, 0], "y": emb_tsne[:, 1], "label": y})
    plot_embedding(df_tsne, title="Digits - t-SNE (pca init)")


if __name__ == "__main__":
    demo_flow()


---

# Dashboard interactivo: `dash_dimred_app.py`

Este archivo permite jugar con parámetros de PCA y t-SNE en un entorno web. Cómo correrlo: `python dash_dimred_app.py`).



In [ ]:
"""
dash_dimred_app.py
Aplicación Dash para reducción de dimensionalidad con PCA y t-SNE
usando el dataset Digits de sklearn.

Autor: [Tu Nombre]
Curso: Técnicas avanzadas en Ciencia de Datos
"""

# ===============================
# Importación de librerías
# ===============================
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
from sklearn.datasets import load_digits
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import pandas as pd

# ===============================
# Carga del dataset
# ===============================
digits = load_digits()
X = digits.data  # variables (64 dimensiones)
y = digits.target  # etiquetas (0–9)

# ===============================
# Inicialización de la aplicación
# ===============================
app = dash.Dash(__name__)
server = app.server  # para despliegue en producción (ej. Heroku)

# ===============================
# Layout de la app
# ===============================
app.layout = html.Div([
    html.H1("Reducción de Dimensionalidad: PCA vs t-SNE", 
            style={"textAlign": "center"}),

    html.Div([
        html.Label("Seleccione técnica de reducción:"),
        dcc.Dropdown(
            id="method-dropdown",
            options=[
                {"label": "PCA", "value": "PCA"},
                {"label": "t-SNE", "value": "TSNE"}
            ],
            value="PCA",
            clearable=False
        )
    ], style={"width": "40%", "margin": "auto"}),

    dcc.Graph(id="scatter-plot", style={"height": "80vh"})
])

# ===============================
# Callbacks
# ===============================
@app.callback(
    Output("scatter-plot", "figure"),
    [Input("method-dropdown", "value")]
)
def update_graph(method):
    """
    Callback para actualizar el gráfico de dispersión según
    la técnica seleccionada (PCA o t-SNE).
    """

    if method == "PCA":
        reducer = PCA(n_components=2)
        reduced_data = reducer.fit_transform(X)
    else:
        reducer = TSNE(n_components=2, random_state=42, perplexity=30)
        reduced_data = reducer.fit_transform(X)

    # Crear DataFrame para Plotly
    df = pd.DataFrame({
        "Component 1": reduced_data[:, 0],
        "Component 2": reduced_data[:, 1],
        "Digit": y
    })

    # Gráfico interactivo
    fig = px.scatter(
        df, x="Component 1", y="Component 2",
        color=df["Digit"].astype(str),
        title=f"Visualización con {method}",
        labels={"Digit": "Etiqueta del dígito"},
        opacity=0.7
    )

    return fig


# ===============================
# Main
# ===============================
if __name__ == "__main__":
    app.run_server(debug=True)


### Cómo usarlo

1. Guarda el archivo como `dash_dimred_app.py`.
2. Instala dependencias si no las tienes:

   ```bash
   pip install dash plotly scikit-learn pandas
   ```
3. Ejecuta:

   ```bash
   python dash_dimred_app.py
   ```
4. Abre en el navegador `http://127.0.0.1:8050`

Tendrás una aplicación donde puedes elegir **PCA** o **t-SNE** en un dropdown y ver el resultado de la reducción de dimensionalidad en un gráfico interactivo.

---
